<a href="https://colab.research.google.com/github/Nadaramadan1/AI_Workshop_Projects/blob/main/Nada_Ramadan_Bone_Fructure_Detecion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("bmadushanirodrigo/fracture-multi-region-x-ray-data")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fracture-multi-region-x-ray-data


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D,BatchNormalization,MaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from IPython.display import SVG, Image
from sklearn.model_selection import train_test_split
import os
import cv2
import random
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img
from PIL import Image, UnidentifiedImageError
import shutil
from shutil import copyfile
import plotly.express as px


# **Explore Data**

In [ ]:
train=path+'/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train'
test=path+'/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/test'
val=path+'/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/val'

In [ ]:
train_dir = path + '/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train'
test_dir  = path + '/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/test'
val_dir   = path + '/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/val'


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='binary')
val_data = val_datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=32, class_mode='binary')
test_data = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode='binary')


Found 9246 images belonging to 2 classes.
Found 829 images belonging to 2 classes.
Found 506 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [ ]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import os
from PIL import Image, UnidentifiedImageError

def remove_corrupted_images(directory):
    num_removed = 0
    for subdir in os.listdir(directory):
        folder_path = os.path.join(directory, subdir)
        for fname in os.listdir(folder_path):
            fpath = os.path.join(folder_path, fname)
            try:
                img = Image.open(fpath)
                img.verify()  # Check for corruption
            except (UnidentifiedImageError, OSError):
                os.remove(fpath)
                num_removed += 1
    print(f"Removed {num_removed} corrupted images.")

# Run it for each split
remove_corrupted_images(train_dir)
remove_corrupted_images(val_dir)
remove_corrupted_images(test_dir)
history = model.fit(train_data, validation_data=val_data, epochs=10)


Removed 0 corrupted images.
Removed 0 corrupted images.
Removed 0 corrupted images.
Epoch 1/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 1324s 5s/step - accuracy: 0.9808 - loss: 0.0684 - val_accuracy: 0.9505 - val_loss: 0.1954
Epoch 2/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 1346s 5s/step - accuracy: 0.9783 - loss: 0.0748 - val_accuracy: 0.9505 - val_loss: 0.2209
Epoch 3/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 1316s 5s/step - accuracy: 0.9820 - loss: 0.0753 - val_accuracy: 0.9650 - val_loss: 0.3687
Epoch 4/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 1326s 5s/step - accuracy: 0.9809 - loss: 0.0692 - val_accuracy: 0.9288 - val_loss: 1.2183
Epoch 5/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 1344s 5s/step - accuracy: 0.9849 - loss: 0.0872 - val_accuracy: 0.9409 - val_loss: 0.3076
Epoch 6/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 1366s 5s/step - accuracy: 0.9876 - loss: 0.0483 - val_accuracy: 0.9614 - val_loss: 0.1990
Epoch 7/10
289/289 ━━━━━━━━━━━━━━━━━━━━ 1321s 5s/step - accuracy: 0.9867 - loss: 0.0569 - val_accuracy: 0.9650 - val_loss: 0.1405
Epoch 

In [ ]:
model.evaluate(test_data)

16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.9896 - loss: 0.0443


[0.02962850220501423, 0.9940711259841919]